In [ ]:
import cv2
import cvzone
import numpy as np
import time
from ultralytics import YOLO
import pygame
import telepot
import os

path = r"D:\Computer Vision\wrong-way-driving-detection-yolov8-wrong-way-vehicle-detection-yolov8-traffic-management--main"
os.chdir(path)


from tracker import Tracker


# Model 
model = YOLO("yolov8s.pt")

# Data
df = open(r"coco.txt" , "r")
classes = df.read().split("\n")

pygame.mixer.init()
pygame.mixer.music.load("alarm.wav")

# Telegram Bot
bot = telepot.Bot(token="7219576373:AAHwOtWOXV4itelGQILKLNIQsVyNyLDhWd8")
chat_id = "5045069081"


def send_photo_to_telegram(image_path):
    bot.sendPhoto(chat_id, open(image_path, 'rb'))

    
cap = cv2.VideoCapture(r"wrongway.mp4")

tracker = Tracker()

car_status = {}

area1 = [(593,227), (602,279), (785,274), (774,220)]
area2 = [(747,92), (804, 302), (842, 295), (773,95)]


while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (1020, 700))
    
    results = model(frame)
    
    lis = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            if int(box.cls[0]) < len(classes):
                name = classes[int(box.cls[0])]
                conf = box.conf[0]
                
                if name == "car" and conf > 0.5:
                    lis.append([x1, y1, x2, y2])
                    bbox_id = tracker.update(lis)
                    
                    for bb in bbox_id:
                        x1, y1, x2, y2, idd = bb
                        
                        cx = int((x1 + x2) // 2)
                        cy = int((y1 + y2) // 2)
                        
                        width = x2 - x1
                        height = y2 - y1
                        
                        cvzone.putTextRect(frame, f"Car, Conf: {conf * 100:.2f}%", 
                                           (x1 - 5, y1 - 15), scale=1, thickness=1,
                                           colorR=(0, 255, 255), colorT=(0, 0, 0))
                        
                        cvzone.cornerRect(frame, (x1, y1, width, height), l=10, t=10, rt=2,
                                          colorR=(0, 0, 0), colorC=(0, 255, 0))
                        
                        cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)
                        
                        
                        result_area1 = cv2.pointPolygonTest(np.array(area1, np.int32), (cx, cy), False) >= 0
                        result_area2 = cv2.pointPolygonTest(np.array(area2, np.int32), (cx, cy), False) >= 0
                        
                        
                        if idd not in car_status:
                            car_status[idd] = {"result_area1": False, "result_area2": False, "wrong_way": False}
                        
                        if result_area1:
                            car_status[idd]["result_area1"] = True
                        
                        if result_area2:
                            car_status[idd]["result_area2"] = True
                         
                        
                        # If the car enters area2 without entering area1 first, mark it as wrong way
                        if car_status[idd]["result_area2"] and not car_status[idd]["result_area1"] and not car_status[idd]["wrong_way"]:
                            car_status[idd]['wrong_way'] = True
                            
                            if car_status[idd]['wrong_way'] :
                                # Only trigger if the wrong way is detected
                                cv2.polylines(frame, [np.array(area2, np.int32)], True, (0, 0, 255), 2)

                                cvzone.cornerRect(frame, (x1, y1, width, height), l=10, t=10, rt=2,
                                              colorR=(0, 0, 255), colorC=(0, 0, 255))

                                cvzone.putTextRect(frame, 'This Car Is Wrong Way!', 
                                                   (x1, y1 + 30), 1, 1, colorR=(0, 0, 255), colorT=(0, 0, 0))

                                if not pygame.mixer.music.get_busy():
                                    pygame.mixer.music.play()

                                image_path = f"Wrong_Way_Driving_Detection.jpg"
                                cv2.imwrite(image_path, frame)
                                send_photo_to_telegram(image_path)



                            
                        # Reset the status if the car correctly moves from area1 to area2
                        if car_status[idd]["result_area1"] and car_status[idd]["result_area2"]:
                            car_status[idd]['wrong_way'] = False

    cv2.polylines(frame, [np.array(area1, np.int32)], True, (0, 255, 0), 2)
    cv2.polylines(frame, [np.array(area2, np.int32)], True, (0, 255, 0), 2)

    cv2.imshow("Wrong Way Driving Detection", frame)
    
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()
